<a href="https://colab.research.google.com/github/Alvorecer721/waymo-motion-prediction-challenge-2022-multipath-plus-plus/blob/boris%2Fnuscenes-configs/code/nuscenes_full_cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install nuscenes-devkit matplotlib==3.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of nuscenes-devkit to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.6/312.6 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=dd33bbc754f658c38f21f9b5674c6f9691c92df7f31599d0a9ee3f724

In [3]:
!git clone https://github.com/Alvorecer721/waymo-motion-prediction-challenge-2022-multipath-plus-plus multipathpp
!cd multipathpp && git checkout boris/nuscenes-configs

import sys
sys.path.insert(0, '/content/multipathpp/code')

Cloning into 'multipathpp'...
remote: Enumerating objects: 556, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 556 (delta 290), reused 314 (delta 219), pack-reused 149
Receiving objects: 100% (556/556), 4.24 MiB | 16.75 MiB/s, done.
Resolving deltas: 100% (366/366), done.
Branch 'boris/nuscenes-configs' set up to track remote branch 'boris/nuscenes-configs' from 'origin'.
Switched to a new branch 'boris/nuscenes-configs'


In [ ]:
!python3 multipathpp/code/prerender/prerender_nuscenes.py \
    --data-version v1.0-mini \
    --data-path drive/MyDrive/multipathpp/nuscenes/v1.0-mini \
    --output-path drive/MyDrive/multipathpp/prerendered_nuscenes \
    --config multipathpp/code/configs/nuscenes_prerender.yaml

2023-05-31 09:50:10.685416: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 09:50:12.387821: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 13.675 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
 50% 5/10 [00:15<00:11,  2.34s/it]

In [ ]:
import os


PRERENDERED_NUSCENES_PATH = 'drive/MyDrive/multipathpp/prerendered_nuscenes'
train_files = [filename for filename in os.listdir(PRERENDERED_NUSCENES_PATH) if int(filename.split('_')[1]) < 8]
val_files = [filename for filename in os.listdir(PRERENDERED_NUSCENES_PATH) if int(filename.split('_')[1]) >= 8]

os.makedirs(os.path.join(PRERENDERED_NUSCENES_PATH, 'train'))
os.makedirs(os.path.join(PRERENDERED_NUSCENES_PATH, 'val'))

for f in train_files:
  os.rename(os.path.join(PRERENDERED_NUSCENES_PATH, f), os.path.join(PRERENDERED_NUSCENES_PATH, 'train', f))

for f in val_files:
  os.rename(os.path.join(PRERENDERED_NUSCENES_PATH, f), os.path.join(PRERENDERED_NUSCENES_PATH, 'val', f))

!ls drive/MyDrive/multipathpp/prerendered_nuscenes

In [ ]:
!python3 multipathpp/code/normalization.py \
    --data-path drive/MyDrive/multipathpp/prerendered_nuscenes/train \
    --output-path drive/MyDrive/multipathpp/normalization/normalization_coefs_nuscenes.npy \
    --config multipathpp/code/configs/nuscenes_normalization.yaml

In [7]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 86.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-kyv1st6r
  Running command git clone --filter=blob:none --quiet https://github.com/py

In [10]:
!python3 multipathpp/code/train.py \
    --train-data-folder drive/MyDrive/multipathpp/prerendered_nuscenes/train \
    --val-data-folder drive/MyDrive/multipathpp/prerendered_nuscenes/val \
    --norm-coeffs drive/MyDrive/multipathpp/normalization/normalization_coefs_nuscenes.npy \
    --config multipathpp/code/configs/nuscenes_final_RoP_Cov_Single.yaml \
    --epoch 1

2023-05-31 09:11:01.038663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 09:11:01.921041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/multipathpp
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Epochs:   0%|          | 0/1 [00:00<?, ?it/s]
Iterations:   0% 0/3 [